In [1]:
## SETUP (OPTIONAL - USE IF REQUIRED)
## Bracketed Paste is disabled to prevent characters after output
## Example: 
## $ echo 'Hii'
## | Hi?2004l
# bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it.
# echo "TODO:"
# echo "  echo \"\$PS1\" | command grep --count --perl-regexp '[\x00-\x1f]'"

actual=${PS1: -1}
[ "$actual" = ">" ]; echo $?

0


In [3]:
# Setting a temp directory for tests
[ "$DEBUG_LEVEL" -lt 4 ]; echo "$?"

0


In [4]:
# Global Setup

## Aliases for censoring username and/or numbers
alias testnum="sed -r "s/[0-9]/N/g"" 
alias testuser="sed -r "s/"$USER"+/userxf333/g""
alias testuserpath="sed 's|/home/[^/]\+/|USER_PATH/|'"
alias linebr="printf '%132s\n' ' ' | tr ' ' '-'"

In [5]:
# Global setup

## Setting a temp directory for tests
## NOTE: If debugging uses $TMP
## Use of rm command is heavily discouraged, use move to $trash_dir instead

TMP=${TMP:-/tmp}
if [ "$DEBUG_LEVEL" -lt 4 ]; then TMP=/tmp/test-grep-commands; fi

## OLD: Used the concept of trash-dir, instead use unique temp_dirs
# temp_dir=$TMP/test-7371
# trash_dir=$TMP/"_temp-trash-$(date "+%Y%m%d%H%M%S")"
# command mkdir -p $temp_dir
# command mkdir -p $trash_dir
# command cd $temp_dir

temp_dir=$TMP/alias-grep-commands
rename-with-file-date "$temp_dir"
command mkdir -p "$temp_dir"
command cd "$temp_dir"

renamed '/tmp/test-grep-commands/alias-grep-commands' -> '/tmp/test-grep-commands/alias-grep-commands.16Apr24.9'


In [6]:
## Count aliases w/ -dir in name

## Example:
##   $ alias | egrep '\-dir\b' | wc -l
##   29

dir_aliases=$(alias | egrep '\-dir\b' | wc -l)
[ "$dir_aliases" -gt 20 ]; echo "$?"

0


In [7]:
## Count functions w/ -dir in name

## Example:
## $ typeset -f | egrep '^\w+' | wc -l
## | 474

dir_functions=$(typeset -f | egrep '\-dir\b' | wc -l)
[ "$dir_functions" -gt 35 ]; echo "$?"

0


In [8]:
## BAD
# source $BIN_DIR/tomohara-aliases.bash

In [9]:
grep_ver=$(grep -V | head -n 1 | tail )
echo $grep_ver

grep (GNU grep) 3.7


In [10]:
# Global Setup

## SETTING UP FOR GREP TEST
## BAD: rm -rf ./* > /dev/null
# Not recommended as not contents in directory: mv -f ./* $trash_dir

# THE WORD TO BE TESTED - sensitive
command mkdir testdir1 testdir2
echo "As grep commands are case sensitive, one of the most useful operators for grep searches as they are sensitive is -i. Instead of printing lowercase results only, the terminal displays both uppercase and lowercase results. The output includes lines with mixed case entries.
" > testgrep1
echo "sensitive sensitive sensitive Sensitive SENSITIVE lorem ipsum" > testgrep2
echo "no mentions here" > testgrep3
echo "Passwords are generally case sensitive" > testgrep4
printf "Non ASCII: Ñ\nNext\nNon ASCII: §" > testgrep5


In [11]:
## TEST-2: TESTING GREP ALIASES

# 0) grep 
grep sensitive testgrep1 testgrep2 testgrep3 testgrep4

## OLD
# linebr
# # A) gu (grep-unique i-)
# gu sensitive testgrep1 testgrep2 testgrep3 testgrep4
# linebr
# # B) gu- (grep-unique)
# gu- sensitive testgrep1 testgrep2 testgrep3 testgrep4
# linebr
# # C) grepl- (grep-to-less)
# grepl- "sensitive" testgrep1 testgrep2 testgrep3 testgrep4 | wc -l
# # NOTE: wc -l is pipelined as command goes to a loop

testgrep1:As grep commands are case sensitive, one of the most useful operators for grep searches as they are sensitive is -i. Instead of printing lowercase results only, the terminal displays both uppercase and lowercase results. The output includes lines with mixed case entries.
testgrep2:sensitive sensitive sensitive Sensitive SENSITIVE lorem ipsum
testgrep4:Passwords are generally case sensitive


In [12]:
# A) gu (grep-unique i-)
gu sensitive testgrep1 testgrep2 testgrep3 testgrep4

testgrep4:1
testgrep2:1
testgrep1:1


In [13]:
# B) gu- (grep-unique)
gu- sensitive testgrep1 testgrep2 testgrep3 testgrep4

testgrep4:1
testgrep2:1
testgrep1:1


In [14]:
# C) grepl- (grep-to-less)
grepl- "sensitive" testgrep1 testgrep2 testgrep3 testgrep4 | wc -l

3


In [15]:
## Halts test
# show-line-context testgrep3

In [16]:
## OLD
cat testgrep2 | gr-less lorem

1:sensitive sensitive sensitive Sensitive SENSITIVE lorem ipsum


In [17]:
cat testgrep5 | grep-nonascii

Non ASCII: Ñ
Non ASCII: §


In [18]:
# Need help for: fs-, findgrep, fgr, fgr-py

In [19]:
prepare-find-files-here | testuser | testnum | awk '!($6="")'
# prepare-find-files-here | sed 's|\([^ ]\+\) \([^ ]\+\) \([^ ]\+\) \([^ ]\+\) \([^ ]\+\) \([^ ]\+\) \([^ ]\+\) \([^ ]\+\) \(.*\)|\1 \6 \7 \8|'

-rw-r--r-- N userxfNNN userxfNNN N  NN NN:NN ./ls-alR.list
-rw-r--r-- N userxfNNN userxfNNN NN  NN NN:NN ./ls-alR.list.log
-rw-r--r-- N userxfNNN userxfNNN N  NN NN:NN ./ls-aR.list
-rw-r--r-- N userxfNNN userxfNNN NN  NN NN:NN ./ls-aR.list.log


In [20]:
command mkdir backup/
ls

backup       ls-alR.list.log  ls-aR.list.log  testdir2   testgrep2  testgrep4
ls-alR.list  ls-aR.list       testdir1        testgrep1  testgrep3  testgrep5


In [21]:
find-files "ls*" | testuser | testnum | awk '!($6="")'

In [22]:
# # Need Help (Non-ending process)
# $ find-files- "testgrep4" | wc -l

In [23]:
# # # ISSUE
# # $ make-file-listing
# | cp: cannot stat 'ls-aR.list': No such file or directory
# | chmod: cannot access './backup/ls-aR.list': No such file or directory
## BAD: command mv -f * $trash_dir

In [24]:
echo "END"

END
